# Pop Grid Compare

Goal of this notebook is to load all availabe popgrid date products with the same spatial resolution, projections, and CRS, stack them, and indentify areas of geographic variance.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio 
import rioxarray as rio

In [2]:
# File paths and FN
data_in = '/Users/cascade/Github/PopGridCompare/data/raw/'
data_out = '/Users/cascade/Github/PopGridCompare/data/interim/'

wp_path = data_in+'WorldPop16/ppp_2016_1km_Aggregated.tif'
ls_path = data_in+'LandScan-Global-2015/lspop2015/w001001.adf'
esri_path = data_in+'ESRI_WPE_2016_Pop/WPE_1KM_2016_Pop.tif'

## Open files and check meta data

In [3]:
# World Pop : width = 43200 height = 18720 epsg = 4326
wp = rio.open_rasterio(wp_path)
wp

<xarray.DataArray (band: 1, y: 18720, x: 43200)>
[808704000 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * y            (y) float64 84.0 83.99 83.98 83.97 ... -71.98 -71.99 -72.0
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
    spatial_ref  int64 0
Attributes:
    transform:     (0.0083333333, 0.0, -180.001249265, 0.0, -0.0083333333, 83...
    _FillValue:    -3.4028234663852886e+38
    scale_factor:  1.0
    add_offset:    0.0
    grid_mapping:  spatial_ref

In [4]:
# ESRI : width = 40074 height = 14285 epsg = 4326
esri = rio.open_rasterio(esri_path)
esri

<xarray.DataArray (band: 1, y: 14285, x: 40074)>
[572457090 values with dtype=int32]
Coordinates:
  * band         (band) int64 1
  * y            (y) float64 73.32 73.31 73.3 73.29 ... -54.98 -54.99 -55.0
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:      336720
    STATISTICS_MEAN:         401.47917136019
    STATISTICS_MINIMUM:      1
    STATISTICS_SKIPFACTORX:  1
    STATISTICS_SKIPFACTORY:  1
    STATISTICS_STDDEV:       1594.7670745165
    transform:               (0.008983196, 0.0, -179.99191516916386, 0.0, -0....
    _FillValue:              -2147483647.0
    scale_factor:            1.0
    add_offset:              0.0
    grid_mapping:            spatial_ref

In [5]:
# LS : width = 43200 height = 21600 epsg = 4326
ls = rio.open_rasterio(ls_path)
ls


<xarray.DataArray (band: 1, y: 21600, x: 43200)>
[933120000 values with dtype=int32]
Coordinates:
  * band         (band) int64 1
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  38537
    STATISTICS_MEAN:     16.167777587702
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   233.963508769
    transform:           (0.0083333333333333, 0.0, -180.0, 0.0, -0.0083333333...
    _FillValue:          -2147483647.0
    scale_factor:        1.0
    add_offset:          0.0
    grid_mapping:        spatial_ref

## Test Matching and Reprojecting

trying: https://corteva.github.io/rioxarray/html/examples/reproject_match.html

In [6]:
# Check NA values
print(esri.attrs['_FillValue'])
print(ls.attrs['_FillValue'])
print(wp.attrs['_FillValue'])

-2147483647.0
-2147483647.0
-3.4028234663852886e+38


In [7]:
# Change NA values
na_val = -9999
esri.attrs['_FillValue'] = na_val
ls.attrs['_FillValue'] = na_val
wp.attrs['_FillValue'] = na_val

In [8]:
# Check NA values
print(esri.attrs['_FillValue'])
print(ls.attrs['_FillValue'])
print(wp.attrs['_FillValue'])

-9999
-9999
-9999


In [11]:
# Match WorldPop/ESRI to LandSan write it out

wp_match = wp.rio.reproject_match(ls)
esri_match = esri.rio.reproject_match(wp)

In [12]:
# Check NA valuesb
print(esri_match.attrs['_FillValue'])
print(wp.attrs['_FillValue'])
print(ls.attrs['_FillValue'])


-9999
-9999
-9999


In [14]:
# Try writing them and then checking them

esri_match.rio.to_raster(data_out+'ESRI16_matched.tif')
wp_match.rio.to_raster(data_out+'WP16_matched.tif')



In [15]:
ls.rio.to_raster(data_out+'LS15_matched.tif')

CPLE_AppDefinedError: TIFFReadDirectory:Cannot handle zero number of tiles

## Subtract 'EM

In [ ]:
# This produces strange results

wp_ls = wp - ls_match
wp_esri = wp - esri_match
ls_esri = ls_match - esri_match

In [ ]:
# write them to files
wp_ls.rio.to_raster(data_out+'wp_ls.tif')
wp_esri.rio.to_raster(data_out+'wp_esri.tif')
ls_esri.rio.to_raster(data_out+'ls_esri.tif')

In [ ]:
rgb_array = xr.concat([wp, esri_match, ls_match], dim = 'band')

In [ ]:
#rgb_array.rio.to_raster(data_out+'all_rgb.tif')